In [1]:
from attack import MagicWordFinder
from sentence_transformers import SentenceTransformer
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = SentenceTransformer("intfloat/e5-base-v2").to(device)

# The positive magic words we find should ideally push arbitrary sentences 
# to be similar to the sentences in S
S = [
"who played cedric in harry potter",
"who is percival graves harry potter",
"who was beatrix potter",
"which is the longest harry potter book",
"who is gilderoy lockhart in harry potter",
"weasley harry potter actor",
"who played draco from harry potter",
"what is the actors name that plays harry potter",
"what year was harry potter released",
"what is the donabe ceramic potter",
"what date what the first harry potter movie released?",
"who played snape in harry potter",
"who is the actor that plays voldemort in harry potter",
"when did harry potter first published",
"who did emma thompson play in harry potter",
"largest harry potter collection",
"who played professor snape in harry potter",
"who does david bradley play in harry potter",
"who is nicholas flamel in harry potter",
"what was the spiders name in harry potter",
"who is the actor for lupin in harry potter",
"what is the name of the phoenix in harry potter",
"what are the names of all harry potter movies",
"who is danny dyer and harry potter",
"definition of transfiguration harry potter",
"how many books in the harry potter series?",
"what year was harry potter born",
"how long does harry potter tour take",
"who is the actor for harry potter",
"who was the stag in harry potter get t",
"when did harry potter open",
"how many chapters are there in harry potter and the order of the phoenix?",
"what is the genre of the harry potter books",
"how many book about harry potter",
"who is cedric in harry potter",
"where does harry potter live",
"when was the first harry potter published",
"when was lily potter born",
"how long is the harry potter movies",
"who played the asian girl in harry potter",
"who played the girl in harry potter",
"how many pages in the harry potter series usa",
"what tickets should we buy for wizarding world of harry potter",
"who plays cedric diggory in harry potter",
"who is harry potter's main antagonist",
"when was the first harry potter books",
"how many nominations does harry potter have",
"what is the name of who played hermione on harry potter",
"when did harry potter and the deathly hallows come",
"when was the harry potter series made",
"what is the actor name who plays harry potter",
"when was harry potter published",
"how old was daniel radcliffe during the harry potter movies",
"number of words pages chapters in harry potter book series",
"when did the first harry potter come out",
"who directed the last three harry potter movies",
"what is the language snake they speak in harry potter",
"what kind of archetype is harry potter",
"is bellatrix lestrange related to harry potter",
"who published harry potter",
"who is scabbers in harry potter"
]

In [2]:
attack = MagicWordFinder(model, S)

# Find 30 candidates for 1-token magic words; the algorithm will only return the 10 best candidates, since we specify k_0=10
cands = attack.find_magic_words(k=30, m=1, k_0=10, epochs=1)

d:\research\gaslite-main\.venv\lib\site-packages\torch\autograd\graph.py:823: UserWarning: Attempting to run cuBLAS, but there was no current CUDA context! Attempting to set the primary context... (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\pytorch\aten\src\ATen\cuda\CublasHandlePool.cpp:180.)
  return Variable._execution_engine.run_backward(  # Calls into the C++ engine to run the backward pass


In [3]:
# Print the magic words found
for i, cand in enumerate(cands):
    print(f"{i + 1}. {model.tokenizer.decode(cand)} (token IDs {cand})")

# Evaluate how the magic words affect the avg. cosine similarity with S
sentence = "Voldemort was right all along!"
sentence_emb = model.encode(sentence, convert_to_tensor=True)
S_embed = model.encode(S, convert_to_tensor=True)

1. mp (token IDs [6131])
2. ##pet (token IDs [22327])
3. ##unced (token IDs [22392])
4. ##uder (token IDs [29190])
5. ##rao (token IDs [25667])
6. ##ir (token IDs [4313])
7. ##dant (token IDs [28210])
8. ##rang (token IDs [24388])
9. ##att (token IDs [19321])
10. ##hu (token IDs [6979])


In [4]:
# Evaluate the base cosine similarity (i.e. with no magic words appended)
cos_sim = torch.nn.CosineSimilarity(dim=0)
avg_cos_sim = sum([cos_sim(sentence_emb, S_embed[j]) for j in range(len(S))]) / len(S)

print(f"Base avg. cosine similarity: {avg_cos_sim}")

Base avg. cosine similarity: 0.7476574778556824


In [5]:
# Compare with the avg. similarity to the centroid of the queries
centroid = model.encode(S, convert_to_tensor=True).mean(dim=0)
avg_cos_sim = sum([cos_sim(centroid, S_embed[j]) for j in range(len(S))]) / len(S)

print(f"Avg. cosine similarity w/centroid: {avg_cos_sim}")

Avg. cosine similarity w/centroid: 0.893332839012146


In [6]:
### Evaluate how appending each magic word once impacts the avg. cosine similarity
sentence_tokenized = model.tokenizer(sentence)["input_ids"]
# Extract the EOS token and remove it (since we append more tokens)
eos_token = sentence_tokenized[-1]
sentence_tokenized = sentence_tokenized[:-1]

for cand in cands:
    # Append the candidate magic word to the sentence and then append the EOS token
    input_ids = sentence_tokenized + cand + [eos_token]
    # Embed the result
    magic_sent_emb = model.encode(model.tokenizer.decode(input_ids), convert_to_tensor=True)
    # Evaluate avg. cosine similarity
    avg_cos_sim = sum([cos_sim(magic_sent_emb, S_embed[j]) for j in range(len(S))]) / len(S)

    print(f"Cosine similarity after appending magic word '{model.tokenizer.decode(cand)}': {avg_cos_sim}")

Cosine similarity after appending magic word 'mp': 0.7553719878196716
Cosine similarity after appending magic word '##pet': 0.7553876638412476
Cosine similarity after appending magic word '##unced': 0.7511067390441895
Cosine similarity after appending magic word '##uder': 0.7551242709159851
Cosine similarity after appending magic word '##rao': 0.7480366826057434
Cosine similarity after appending magic word '##ir': 0.7548622488975525
Cosine similarity after appending magic word '##dant': 0.7531682252883911
Cosine similarity after appending magic word '##rang': 0.7553225159645081
Cosine similarity after appending magic word '##att': 0.7531187534332275
Cosine similarity after appending magic word '##hu': 0.7577243447303772
